### Training Loop

- the goal of callbacks in FastAI is to **allow the training loop to be highly customizable** 
- this is accomplished by injecting conditions such as `on_batch_begin`, `on_batch_end`, `on_epoch_begin`, `on_epoch_end` etc that trigger corresponding custom callback functions

#### Basic Mini-Batch Training Loop

In [ ]:
for epoch in range(epochs):
    learn.model.train() # set to training mode
    for xb,yb in learn.data.train_dl: # grabs mini-batch from dataloader
        loss = learn.loss_func(learn.model(xb), yb) # compute loss
        loss.backward()  # compute gradients
        learn.opt.step() # update parameters
        learn.opt.zero_grad() # reset gradients

#### The "infinitely customizable" training loop

The code snippet below demonstrates the scafold of a training loop that allows a wide variety of customizability based on callbacks.

In [ ]:
callbacks.on_train_begin()
for _ in range(epoch):
    callbacks.on_epoch_begin()  #---------------callback before an epoch begins
    learn.model.train()
    for xb,yb in learn.data.train_dl:
        callbacks.on_batch_begin() #------------callback before a batch begins
        y = model(xb)
        callbacks.on_loss_begin()  #------------callback before loss computation begins
        loss = loss_func(y, yb)
        callbacks.on_backward_begin() #---------callback before gradient computation begin
        loss.backward() # backward pass
        callbacks.on_backward_end()  #----------callback after gradient computation begin
        opt.step()
        callbacks.on_step_end() #---------------callback after param update begins
        opt.zero_grad() 
        callbacks.on_batch_end() #--------------callback after a batch
    callbacks.on_epoch_end() #------------------callback after an epoch
callbacks.on_train_end() #----------------------callback after training

### Callback class

To provide the generic functionality 